### **MNIST**

### **LIBRERIE & IMPORT**

In [6]:
! pip install -q tensorflow

In [7]:
# Librerie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import tensorflow

################ SOLO SU VS CODE ##################
#Funzione di plotting
# import os, sys
#sys.path.append(os.path.abspath(".."))
# from FUNCTIONS.utils import <nome-funzione>
###################################################

# Funzione che serve ad effettuare la divisione tra train set e test set
from sklearn.model_selection import train_test_split
# Metriche per definire la qualità del modello generato
from sklearn.metrics import accuracy_score #, mean_squared_error, classification_report
# Caricamento Dati
from sklearn.datasets import fetch_openml
from sklearn.datasets import load_digits
# Per Scaling dei dati
from sklearn.preprocessing import StandardScaler
# Decision Tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# SVM
from sklearn.svm import SVC
# KNN
from sklearn.neighbors import KNeighborsClassifier
# K-MEANS
from sklearn.cluster import KMeans
# NEURAL NETWORK RELU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

### **DATASET MNIST**

In [9]:

RANDOM_SEED = 42

# Versione completa, 28x28
mnist = fetch_openml('mnist_784')
# Versione ridotta, 8x8
# mnist = load_digits()

X, y = mnist.data, mnist.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_SEED)

# Normalizzo i dati applicando lo scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### **ACCURACY**

In [4]:
classifiers = {
    "Decision Tree" : DecisionTreeClassifier(),
    "Regressione Logistica" : LogisticRegression(max_iter=10000),
    "SVM" : SVC(),
    "KNN" : KNeighborsClassifier()
}

for name, model in classifiers.items():
    start = time.time()
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    end = time.time()
    accuracy_mnist = accuracy_score(y_test, y_pred)
    print(f"{name} accuracy: {accuracy_mnist*100:.2f}% ({end-start:.2f}s)")

Decision Tree accuracy: 87.27% (25.38s)
Regressione Logistica accuracy: 91.54% (65.17s)
SVM accuracy: 96.30% (675.94s)
KNN accuracy: 94.58% (77.28s)


### **RETE NEURALE - MNIST**

In [10]:
y_train_cat = to_categorical(y_train, num_classes=10)
y_test_cat = to_categorical(y_test, num_classes=10)

model = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation="relu"),
    Dropout(0.2),
    Dense(10, activation="softmax")
])

start = time.time()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train_scaled, y_train_cat, epochs=10, batch_size=32, verbose=0)
loss, nn_accuracy = model.evaluate(X_test_scaled, y_test_cat, verbose=0)
end = time.time()
print(f"Neural Network accuracy: {nn_accuracy*100:.2f}% ({end-start:.2f}s)")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 784), dtype=float32). Expected shape (None, 28, 28), but input has incompatible shape (32, 784)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 784), dtype=float32)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>

### **CONSIDERAZIONI**

**optimizer="adam"** => Algoritmo di ottimizzazione che aggiorna i pesi durante l'allenamento (Adam è molto efficiente)
\
**loss="categorical_crossentropy"** => Funzione di perdita per problemi di classificazione multi-classe (abbiamo 10 classi)
\
**metrics=["accuracy"]** => Metrica da monitorare durante l'allenamento (visualizzi l'accuracy ad ogni epoca)
\

**1. to_categorical()**    → Trasforma label in one-hot (3 → [0,0,0,1,0,0,0,0,0,0])
\
**2. Flatten()**           → Converte 28x28 in vettore di 784
\
**3. Dense(128, relu)**    → Hidden layer con 128 neuroni
\
**4. Dropout(0.2)**        → Disattiva 20% per evitare overfitting
\
**5. Dense(10, softmax)**  → Output con 10 neuroni (probabilità per ogni cifra)
\
**6. compile()**           → Configura optimizer, loss, metrics
\
**7. fit()**               → Allena il modello per 10 epochs
\
**8. evaluate()**          → Valuta su dati di test

**Dense** => layer connessi
\
**Dropout** => previene overfitting
\
**compile()** => configura come allenare

**Due Dense:** *uno elabora (hidden) e uno classifica (output)*
